In [1]:
import requests
from scrapy.http import TextResponse
import pandas as pd
import datetime

In [2]:
title_ls,link_ls = [],[]
for page in range(1,21+1):
    url = 'http://m.finance.daum.net/m/news/news_list.daum?type=main&page={}'.format(page)
    req = requests.get(url)
    response = TextResponse(req.url , body=req.text , encoding='utf-8')
    for content in range(1,10+1):
        try : 
            title_ls.append(response.xpath('//*[@id="mArticle"]/div[2]/ul/li[{}]/a/strong/text()'.format(content)).extract()[0])
            link_ls.append(response.xpath('//*[@id="mArticle"]/div[2]/ul/li[{}]/a/@href'.format(content)).extract()[0])
        except : pass

In [3]:
len(title_ls) == len(link_ls)

True

In [4]:
df = pd.DataFrame()
df['title'] = title_ls
df['link'] = link_ls
df['link'] = ['http://m.finance.daum.net' + i for i in df['link']]
df.tail()

,title,link
184,"[앵커의 눈] ""수수료 때문에 못살겠다"" vs ""인하 여력 없다""..정부는 '진퇴양난'",http://m.finance.daum.net/m/news/news_content....
185,영국 3분기 임금 3.2% 증가..금융위기 후 최고,http://m.finance.daum.net/m/news/news_content....
186,"OPEC, 2019년 석유 공급과잉 경고..감산 바닥다지기?",http://m.finance.daum.net/m/news/news_content....
187,또 감원 바람 부나.. 조선업계 뒤숭숭,http://m.finance.daum.net/m/news/news_content....
188,"李총리 ""캐나다 세이프가드 조치 우려..양국간 공정무역 지속""",http://m.finance.daum.net/m/news/news_content....


In [5]:
content_ls = []

for i in df['link'].values:
    req = requests.get(i)
    response = TextResponse(req.url , body=req.text , encoding='utf-8')
    content_ls.append(','.join(response.xpath('//*[@id="dmcfContents"]/section/p/text()').extract()).replace(',',''))

In [6]:
len(title_ls) == len(link_ls) == len(content_ls)

True

In [7]:
df['content'] = content_ls
df['content'] = [i.replace("\t",'').replace('\n','') for i in df['content'].values]
df.tail()

,title,link,content
184,"[앵커의 눈] ""수수료 때문에 못살겠다"" vs ""인하 여력 없다""..정부는 '진퇴양난'",http://m.finance.daum.net/m/news/news_content....,[앵커] 가맹점들이 내는 카드수수료는 연매출액에 따라 이렇게 세 구간으로 나뉘어있습...
185,영국 3분기 임금 3.2% 증가..금융위기 후 최고,http://m.finance.daum.net/m/news/news_content....,(런던=연합뉴스) 박대한 특파원 = 영국의 3분기 임금 상승률이 2008년 금융위기...
186,"OPEC, 2019년 석유 공급과잉 경고..감산 바닥다지기?",http://m.finance.daum.net/m/news/news_content....,(서울=뉴스1) 이창규 기자 = 석유수출국기구(OPEC)는 13일 세계 경제가 둔화...
187,또 감원 바람 부나.. 조선업계 뒤숭숭,http://m.finance.daum.net/m/news/news_content....,연말을 앞두고 조선업계에 또다시 ‘구조조정’ 바람이 불 조짐이다. 국내 조선사 ‘빅...
188,"李총리 ""캐나다 세이프가드 조치 우려..양국간 공정무역 지속""",http://m.finance.daum.net/m/news/news_content....,(서울=뉴스1) 박승주 기자 = 이낙연 국무총리는 13일 오후 정부서울청사에서 조지...


In [8]:
df.to_csv('{}_with_link_daum.csv'.format(str(datetime.datetime.now().month) + str(datetime.datetime.now().day),index=False),index=False)